In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Loading Data

In [ ]:
BASE_DIR = Path("/kaggle/input/playground-series-s3e2/")

train = pd.read_csv(BASE_DIR / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_DIR / "test.csv").drop(columns="id")

# Preprocessing

In [ ]:
df = pd.concat([train.drop(columns=["stroke"]), test], axis=0).reset_index(drop=True)
df.head()

# Feature Engineering

In [ ]:
def diabetes_indicator(avg_glucose_level):
    if avg_glucose_level <= 99:
        return "normal"
    elif avg_glucose_level >= 100 and avg_glucose_level <= 125:
        return "prediabetic"
    elif avg_glucose_level <= 200:
        return "type1"
    else:
        return "type2"

df.avg_glucose_level.map(diabetes_indicator).value_counts()

In [ ]:
df["diabetes"] = df.avg_glucose_level.map(diabetes_indicator)
df.head(3)

# One hot encoding

In [ ]:
df = pd.get_dummies(df)
df.head(3)

# Preprocessing v2

In [ ]:
X = df.iloc[:-len(test), :]
test_new = df.iloc[-len(test):, :]
y = train.stroke

In [ ]:
xgb_params = {'n_estimators': 150,
                 'max_depth': 4,
                 'learning_rate': 0.10513333456836436,
                 'min_child_weight': 8,
                 'gamma': 0.002126415095687836,
                 'subsample': 0.5470092690254375,
                 'colsample_bytree': 0.6296509957360737,
                 'reg_alpha': 0.0003558325520757654,
                 'reg_lambda': 0.6330475930599759}

In [ ]:
xgb_model = xgb.XGBClassifier(**xgb_params)

In [ ]:
xgb_model.fit(X, y)

In [ ]:
y_pred = xgb_model.predict_proba(test_new)[:, 1]
y_pred

In [ ]:
# since we dropped id column from our test dataframe earlier but we need it for submission file so we're reloading it
test_idx = pd.read_csv(BASE_DIR / "test.csv").id
test_idx

In [ ]:
submission = pd.DataFrame(data={"id": test_idx, "stroke": y_pred})
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)